In [0]:
!pip install kaggle

In [127]:
!pip install pandas==0.21.1

     |████████████████████████████████| 26.2MB 50.9MB/s 
ERROR: xarray 0.15.1 has requirement pandas>=0.25, but you'll have pandas 0.21.1 which is incompatible.
ERROR: seaborn 0.10.0 has requirement pandas>=0.22.0, but you'll have pandas 0.21.1 which is incompatible.
ERROR: plotnine 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.21.1 which is incompatible.
ERROR: mizani 0.6.0 has requirement pandas>=0.25.0, but you'll have pandas 0.21.1 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.21.1 which is incompatible.
ERROR: featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.21.1 which is incompatible.
ERROR: fbprophet 0.6 has requirement pandas>=0.23.4, but you'll have pandas 0.21.1 which is incompatible.
  Found existing installation: pandas 1.0.3
    Uninstalling pandas-1.0.3:
      Successfully uninstalled pandas-1.0.3


In [0]:
scikit-learn

# Module importing

In [0]:
import pandas as pd
import numpy as np

In [0]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import NearestNeighbors
from multiprocessing import Pool

In [0]:
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold

In [0]:
from sklearn.preprocessing import scale

In [0]:
import tqdm

In [0]:
import re

In [0]:
from bs4 import BeautifulSoup

# Data loading

In [6]:
import os
os.environ['KAGGLE_USERNAME'] = "kryvokhyzha" # username from the json file
os.environ['KAGGLE_KEY'] = "b30f0451d14f9a4f8a939d2fbadf03e8" # key from the json file
!kaggle competitions download -c company-acceptance-prediction # api copied from kaggle

 95% 171M/180M [00:01<00:00, 112MB/s]
100% 180M/180M [00:01<00:00, 129MB/s]
  0% 0.00/167k [00:00<?, ?B/s]
100% 167k/167k [00:00<00:00, 150MB/s]
 79% 65.0M/82.3M [00:00<00:00, 72.3MB/s]
100% 82.3M/82.3M [00:00<00:00, 121MB/s] 


# Data serialization

In [0]:
train_df = pd.read_pickle('train_df.pkl.zip')

In [0]:
test_df = pd.read_pickle('test_df.pkl.zip')

In [9]:
test_df.head(5)

,id,html,text,keywords,accepted_function,rejected_function,accepted_product,rejected_product
0,071c2e15-5fd0-4e7d-b78d-26b7c34491e1,"[b'<!DOCTYPE html>\r\n<html id=""ctl00_htmlMast...",[We use cookies to tailor our service and ensu...,"[technologies, descriptive, mapping, learning,...","IT support, IT managed services, IT integratio...","manufacture, production, processing, distribut...",,
1,e534df73-b385-4a12-8368-38992d3af7d3,"[b'\n<!DOCTYPE html>\n\n<html lang=""en-US"">\n\...",[About us\nAbout Powernet\nPartnerships\nCusto...,"[outsourcing, projects, IT, sensors, convergen...","software development, database development, sy...","wholesale, distribution, retail selling, manuf...",,
2,4e38c9ff-bffe-4251-b2b2-94777c7344fe,[b'<!--\n.----------------. .----------------...,[Work\nApproach\nPeople\nValues\nBlog\nCareers...,"[Inventing, products, risk, development, servi...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,
3,a7f8870c-30f8-4851-8f8b-4c16942b397f,[b'<!DOCTYPE html>\n<!--[if lt IE 7]><html cla...,[Menu\nMenu\nSupport\nInfrastructure\nColocati...,"[low, chain, portfolio, services, Procurement,...","Investment research and information services, ...","Construction, production and supply, Advertisi...",,
4,c062f411-ae3e-486a-9cba-2267889adb2b,"[b'<!DOCTYPE html><html lang=""en-US""><head><me...",[Français (French)\nEnglish (English)\n日本語 (Ja...,"[recruitment, producing, Evaluation, Payroll, ...","general accounting, auditing, book keeping act...","manufacture, production, processing, distribut...",,


# Data preprocessing

## Dropping byte-like parts

In [0]:
# :(

## Extract comments

In [0]:
# :(

## Extract meta information

In [0]:
train_df['Title'] = np.nan
train_df['Keyword'] = np.nan
train_df['Locale'] = np.nan
train_df['Description'] = np.nan

In [13]:
for idx, pages in tqdm.tqdm(enumerate(train_df.html)):
  title_res = []
  keywords_res = []
  locale_res = []
  description_res = []
  for page in pages:
    soup = BeautifulSoup(page, "lxml")  

    res = ''
    for tag in soup.find_all("meta", property='og:title'):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      title_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", attrs={'name':'keywords'}):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      keywords_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", property='og:locale'):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' ' 
      except:
        pass
    if res != '':
      locale_res.append(res.strip()) 

    res = ''
    for tag in soup.find_all("meta", attrs={'name':'description'}):
      try:
        if tag['content'] not in (', ', '', ',', ' ', '.'):
          res += tag['content'].strip() + ' '
      except:
        pass
    if res != '':
      description_res.append(res.strip()) 


  train_df.loc[idx, 'Title'] = str(title_res) if title_res != [] else np.nan
  train_df.loc[idx, 'Keyword'] = str(keywords_res) if keywords_res != [] else np.nan
  train_df.loc[idx, 'Locale'] = str(locale_res) if locale_res != [] else np.nan
  train_df.loc[idx, 'Description'] = str(description_res) if description_res != [] else np.nan


15515it [35:07,  7.36it/s]


### Locale preprocessing

In [0]:
# sl - seldom, ot - other
train_df.Locale = train_df.Locale.fillna('["ot').apply(lambda x: x[2:4]).map({'ot': 'ot', 'en': 'en', 'ja': 'ja', 'pl':'pl',
                                                                              'es': 'sl', 'de': 'sl', 'it': 'sl',
                                                                              'fr': 'sl', 'bg': 'sl', 'zh': 'sl',
                                                                              'sv': 'sl','nl': 'sl','fi': 'sl',
                                                                              'nb': 'sl'})

### Title preprocessing

In [0]:
train_df.Title = train_df.Title.fillna('without_title').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

### Keyword preprocessing

In [0]:
train_df.Keyword = train_df.Keyword.fillna('without_keyword').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

### Description preprocessing

In [17]:
train_df.Description[95]

"['RAFFIN MEDICAL is a manufacturer and a distributor of single-use medical devices for health care professionals, since 1914.', 'Since 1914, Raffin Medical has been developing, manufacturing and commercializing single use medical devices for health care professionals']"

In [0]:
train_df.Description = train_df.Description.fillna('without_description').apply(lambda x: ' '.join(list(set(re.sub(r'[^\w\s\d,\[\]]+', '', x).strip('][').lower().split(', ')))))

In [0]:
train_df.Description = train_df.Description.apply(lambda x: x if re.sub(r'[a-zA-Z0-9\s\-_]+', '', x.strip()) == '' else 'wrong_description')

## Keywords (model base) preproccesing

In [0]:
train_df.keywords = train_df.keywords.apply(lambda x: [re.sub(r'[^\w\s\d]+', ' ', i).lower() for i in x] if x != [] else ['empty_keyword_list'])

## Accepted/rejected preproccesing

In [0]:
train_df.accepted_function = train_df.accepted_function.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.rejected_function = train_df.rejected_function.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.accepted_product = train_df.accepted_product.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

In [0]:
train_df.rejected_product = train_df.rejected_product.apply(lambda x: re.sub(r'[^\w\s\d,]+', ' ', x).lower() if x != '' else x)

## Replace NaN like values

In [0]:
train_df = train_df.replace(['', '_nan_'], [np.nan, np.nan])

## Filling keyword and description

In [0]:
train_df.Keyword = train_df.Keyword.fillna('without_keyword')

In [0]:
train_df.Description = train_df.Description.fillna('without_description')

## Combine all texts and html pages

In [0]:
# :(

# EDA

In [469]:
print(train_df.text[15][0]) #1439

This page is being redirected. If it does not load, click https://www.fscables.com/featured_page/antennax--the-ultra-low-loss-coaxial-cable-range.html



In [464]:
print(str(train_df.html[2028][0]))

b"<br />\n<b>Fatal error</b>:  Uncaught Error: Call to undefined function mcrypt_create_iv() in /home/forumtna/public_html/system/library/encryption.php:8\nStack trace:\n#0 /home/forumtna/public_html/index.php(215): Encryption-&gt;__construct('8751d0dc1cb676e...')\n#1 {main}\n  thrown in <b>/home/forumtna/public_html/system/library/encryption.php</b> on line <b>8</b><br />\n"


In [11]:
train_df['target'].value_counts() / train_df['target'].value_counts().sum()

1    0.744892
2    0.190139
0    0.064969
Name: target, dtype: float64

# Feature generation

In [0]:
train_df['html_page_cnt'] = train_df.html.apply(lambda x: len(x)).astype(np.int16)

## Mean number of rows in texts

In [0]:
train_df['text_rows_mean'] = train_df.text.apply(lambda x: np.sum([len(i.split('\n')) for i in x]) // len(x)).astype(np.int16)

## Missing values columns

In [0]:
train_df['is_mis_accepted_function'] = pd.isnull(train_df.accepted_function).map({True:1, False:0})

In [0]:
train_df['is_mis_rejected_function'] = pd.isnull(train_df.rejected_function).map({True:1, False:0})

In [0]:
train_df['is_mis_accepted_product'] = pd.isnull(train_df.accepted_product).map({True:1, False:0})

In [0]:
train_df['is_mis_rejected_product'] = pd.isnull(train_df.rejected_product).map({True:1, False:0})

In [0]:
train_df['is_mis_keyword'] = train_df['keywords'].apply(lambda x: 1 if x == ['empty_keyword_list'] else 0)

In [0]:
train_df = train_df.fillna('is_missing')

## Redirect feature

In [0]:
train_df['have_error'] = train_df.text.apply(lambda x: 1 if (('redirected' in x[0].lower() or 'error' in x[0].lower()) and len(x) == 1) else 0)

## Similarity criteria features (in accepted and rejected)

In [0]:
acc_f_rej_f_sim = []
for row in train_df.itertuples():
  accepted_f = set(row.accepted_function.replace(',', '').split(' '))
  rejected_f = set(row.rejected_function.replace(',', '').split(' '))
  acc_f_rej_f_sim.append(len(accepted_f.intersection(rejected_f)) / len(accepted_f.union(rejected_f)))

train_df['acc_f_rej_f_sim'] = acc_f_rej_f_sim

In [0]:
acc_p_rej_p_sim = []
for row in train_df.itertuples():
  accepted_p = set(row.accepted_product.replace(',', '').split(' '))
  rejected_p = set(row.rejected_product.replace(',', '').split(' '))
  acc_p_rej_p_sim.append(len(accepted_p.intersection(rejected_p)) / len(accepted_p.union(rejected_p)))

train_df['acc_p_rej_p_sim'] = acc_p_rej_p_sim

In [0]:
kwd_acc_p_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  accepted_p = set(row.accepted_product.replace(',', '').split(' '))
  kwd_acc_p_sim.append(len(kwd.intersection(accepted_p)) / len(kwd.union(accepted_p)))

train_df['kwd_acc_p_sim'] = kwd_acc_p_sim

In [0]:
kwd_rej_p_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  rejected_p = set(row.rejected_product.replace(',', '').split(' '))
  kwd_rej_p_sim.append(len(kwd.intersection(rejected_p)) / len(kwd.union(rejected_p)))

train_df['kwd_rej_p_sim'] = kwd_rej_p_sim

In [0]:
kwd_acc_f_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  accepted_f = set(row.accepted_function.replace(',', '').split(' '))
  kwd_acc_f_sim.append(len(kwd.intersection(accepted_f)) / len(kwd.union(accepted_f)))

train_df['kwd_acc_f_sim'] = kwd_acc_f_sim

In [0]:
kwd_rej_f_sim = []
for row in train_df.itertuples():
  kwd = set(', '.join(row.keywords).replace(',', '').split(' '))
  rejected_f = set(row.rejected_function.replace(',', '').split(' '))
  kwd_rej_f_sim.append(len(kwd.intersection(rejected_f)) / len(kwd.union(rejected_f)))

train_df['kwd_rej_f_sim'] = kwd_rej_f_sim

In [0]:
train_df['dif_kwd_acc_p_rej_p'] = train_df['kwd_acc_p_sim'] - train_df['kwd_rej_p_sim']
train_df['dif_kwd_acc_j_rej_j'] = train_df['kwd_acc_f_sim'] - train_df['kwd_rej_f_sim']

## All accepted and rejected fraction

In [0]:
all_accepted_function = set()
all_rejected_function = set()

all_accepted_product = set()
all_rejected_product = set()

In [0]:
for row in train_df.itertuples():
  all_accepted_function = all_accepted_function.union(set(row.accepted_function.split(', ')))
  all_rejected_function = all_rejected_function.union(set(row.rejected_function.split(', ')))
  all_accepted_product = all_accepted_product.union(set(row.accepted_product.split(', ')))
  all_rejected_product = all_rejected_product.union(set(row.rejected_product.split(', ')))

In [0]:
frc_acc_f = []
frc_rej_f = []
frc_acc_p = []
frc_rej_p = []

for row in train_df.itertuples():
  frc_acc_f.append(len(set(row.accepted_function.split(', '))) / len(all_accepted_function))
  frc_rej_f.append(len(set(row.rejected_function.split(', '))) / len(all_rejected_function))

  frc_acc_p.append(len(set(row.accepted_product.split(', '))) / len(all_accepted_product))
  frc_rej_p.append(len(set(row.rejected_product.split(', '))) / len(all_rejected_product))

train_df['frc_acc_f'] = frc_acc_f
train_df['frc_rej_f'] = frc_rej_f
train_df['frc_acc_p'] = frc_acc_p
train_df['frc_rej_p'] = frc_rej_p

## Accepted and rejected fraction (target=0)

In [0]:
train_df['frc_acc_f_0'] = 0
train_df['frc_rej_f_0'] = 0
train_df['frc_acc_p_0'] = 0
train_df['frc_rej_p_0'] = 0

In [0]:
all_accepted_function_0 = set()
all_rejected_function_0 = set()

all_accepted_product_0 = set()
all_rejected_product_0 = set()

In [0]:
for row in train_df.itertuples():
  if row.target != 0:
    continue
  all_accepted_function_0 = all_accepted_function_0.union(set(row.accepted_function.split(', ')))
  all_rejected_function_0 = all_rejected_function_0.union(set(row.rejected_function.split(', ')))
  all_accepted_product_0 = all_accepted_product_0.union(set(row.accepted_product.split(', ')))
  all_rejected_product_0 = all_rejected_product_0.union(set(row.rejected_product.split(', ')))

In [0]:
for idx, row in enumerate(train_df.itertuples()):
  if row.target != 0:
    continue
  train_df.loc[idx, 'frc_acc_f_0'] = len(set(row.accepted_function.split(', '))) / len(all_accepted_function_0)
  train_df.loc[idx, 'frc_rej_f_0'] = len(set(row.rejected_function.split(', '))) / len(all_rejected_function_0)

  train_df.loc[idx, 'frc_acc_p_0'] = len(set(row.accepted_product.split(', '))) / len(all_accepted_product_0)
  train_df.loc[idx, 'frc_rej_p_0'] = len(set(row.rejected_product.split(', '))) / len(all_rejected_product_0)

## Accepted and rejected fraction (target=1)

In [0]:
all_accepted_function_1 = set()
all_rejected_function_1 = set()

all_accepted_product_1 = set()
all_rejected_product_1 = set()

In [0]:
for row in train_df.itertuples():
  if row.target != 1:
    continue
  all_accepted_function_1 = all_accepted_function_1.union(set(row.accepted_function.split(', ')))
  all_rejected_function_1 = all_rejected_function_1.union(set(row.rejected_function.split(', ')))
  all_accepted_product_1 = all_accepted_product_1.union(set(row.accepted_product.split(', ')))
  all_rejected_product_1 = all_rejected_product_1.union(set(row.rejected_product.split(', ')))

In [0]:
for idx, row in enumerate(train_df.itertuples()):
  if row.target != 1:
    continue
  train_df.loc[idx, 'frc_acc_f_1'] = len(set(row.accepted_function.split(', '))) / len(all_accepted_function_1)
  train_df.loc[idx, 'frc_rej_f_1'] = len(set(row.rejected_function.split(', '))) / len(all_rejected_function_1)

  train_df.loc[idx, 'frc_acc_p_1'] = len(set(row.accepted_product.split(', '))) / len(all_accepted_product_1)
  train_df.loc[idx, 'frc_rej_p_1'] = len(set(row.rejected_product.split(', '))) / len(all_rejected_product_1)

## Accepted and rejected fraction (target=2)

In [0]:
all_accepted_function_2 = set()
all_rejected_function_2 = set()

all_accepted_product_2 = set()
all_rejected_product_2 = set()

In [0]:
for row in train_df.itertuples():
  if row.target != 2:
    continue
  all_accepted_function_2 = all_accepted_function_2.union(set(row.accepted_function.split(', ')))
  all_rejected_function_2 = all_rejected_function_2.union(set(row.rejected_function.split(', ')))
  all_accepted_product_2 = all_accepted_product_2.union(set(row.accepted_product.split(', ')))
  all_rejected_product_2 = all_rejected_product_2.union(set(row.rejected_product.split(', ')))

In [0]:
for idx, row in enumerate(train_df.itertuples()):
  if row.target != 2:
    continue
  train_df.loc[idx, 'frc_acc_f_2'] = len(set(row.accepted_function.split(', '))) / len(all_accepted_function_2)
  train_df.loc[idx, 'frc_rej_f_2'] = len(set(row.rejected_function.split(', '))) / len(all_rejected_function_2)

  train_df.loc[idx, 'frc_acc_p_2'] = len(set(row.accepted_product.split(', '))) / len(all_accepted_product_2)
  train_df.loc[idx, 'frc_rej_p_2'] = len(set(row.rejected_product.split(', '))) / len(all_rejected_product_2)

## Dropping HTML

In [0]:
train_df = train_df.drop(['html'], axis=1)

## KNN features

In [0]:
class NearestNeighborsFeats(BaseEstimator, ClassifierMixin):
    '''
        This class should implement KNN features extraction 
    '''
    def __init__(self, n_jobs, k_list, metric, n_classes=None, n_neighbors=None, eps=1e-6):
        self.n_jobs = n_jobs
        self.k_list = k_list
        self.metric = metric
        
        if n_neighbors is None:
            self.n_neighbors = max(k_list) 
        else:
            self.n_neighbors = n_neighbors
            
        self.eps = eps        
        self.n_classes_ = n_classes
    
    def fit(self, X, y):
        '''
            Set's up the train set and self.NN object
        '''
        # Create a NearestNeighbors (NN) object. We will use it in `predict` function 
        self.NN = NearestNeighbors(n_neighbors=max(self.k_list), 
                                      metric=self.metric, 
                                      n_jobs=1, 
                                      algorithm='brute' if self.metric=='cosine' else 'auto')
        self.NN.fit(X)
        
        # Store labels 
        self.y_train = y
        
        # Save how many classes we have
        self.n_classes = np.unique(y).shape[0] if self.n_classes_ is None else self.n_classes_
        
        
    def predict(self, X):       
        '''
            Produces KNN features for every object of a dataset X
        '''
        if self.n_jobs == 1:
            test_feats = []
            for i in range(X.shape[0]):
                test_feats.append(self.get_features_for_one(X[i:i+1]))
        else:
            '''
                 *Make it parallel*
                     Number of threads should be controlled by `self.n_jobs`  
                     
                     
                     You can use whatever you want to do it
                     For Python 3 the simplest option would be to use 
                     `multiprocessing.Pool` (but don't use `multiprocessing.dummy.Pool` here)
                     You may try use `joblib` but you will most likely encounter an error, 
                     that you will need to google up (and eventually it will work slowly)
                     
                     For Python 2 I also suggest using `multiprocessing.Pool` 
                     You will need to use a hint from this blog 
                     http://qingkaikong.blogspot.ru/2016/12/python-parallel-method-in-class.html
                     I could not get `joblib` working at all for this code 
                     (but in general `joblib` is very convenient)
                     
            '''
            test_feats =[]
            
            pool = Pool(processes=self.n_jobs) 
            for i in range(X.shape[0]):
                test_feats.append(self.get_features_for_one(X[i:i+1]))
                #test_feats.append(pool.apply_async(self.get_features_for_one, (X[i:i+1],)))
          
            test_feats = [res.get() for res in test_feats]
        return np.vstack(test_feats)
        
        
    def get_features_for_one(self, x):
        '''
            Computes KNN features for a single object `x`
        '''

        NN_output = self.NN.kneighbors(x)
        
        # Vector of size `n_neighbors`
        # Stores indices of the neighbors
        neighs = NN_output[1][0]
        
        # Vector of size `n_neighbors`
        # Stores distances to corresponding neighbors
        neighs_dist = NN_output[0][0] 

        # Vector of size `n_neighbors`
        # Stores labels of corresponding neighbors
        neighs_y = self.y_train[neighs] 
        
        ## ========================================== ##
        ##              YOUR CODE BELOW
        ## ========================================== ##
        
        # We will accumulate the computed features here
        # Eventually it will be a list of lists or np.arrays
        # and we will use np.hstack to concatenate those
        return_list = [] 
        
        
        ''' 
            1. Fraction of objects of every class.
               It is basically a KNNСlassifiers predictions.

               Take a look at `np.bincount` function, it can be very helpful
               Note that the values should sum up to one
        '''
        for k in self.k_list:
            feats = np.bincount(neighs_y[:k],minlength=self.n_classes)
            feats  = feats / feats.sum()
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        '''
            2. Same label streak: the largest number N, 
               such that N nearest neighbors have the same label.
               
               What can help you: `np.where`
        '''
        
        if (neighs_y != neighs_y[0]).astype(int).sum() > 0:
            feats = np.where(np.cumsum((neighs_y != neighs_y[0]).astype(int)) == 1)[0][0]
        else:
            feats = len(neighs_y)
            
        feats = [feats]
        
        assert len(feats) == 1
        return_list += [feats]
        
        '''
            3. Minimum distance to objects of each class
               Find the first instance of a class and take its distance as features.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.

               `np.where` might be helpful
        '''
        feats = []
        for c in range(self.n_classes):
            feat = neighs_dist[neighs_y == c][0] if (neighs_y == c).sum() > 0 else 999
            feats.append(feat)
        
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            4. Minimum *normalized* distance to objects of each class
               As 3. but we normalize (divide) the distances
               by the distance to the closest neighbor.
               
               If there are no neighboring objects of some classes, 
               Then set distance to that class to be 999.
               
               Do not forget to add self.eps to denominator.
        '''
        feats = []
        for c in range(self.n_classes):
            feat = neighs_dist[neighs_y == c][0] if (neighs_y == c).sum() > 0 else 999
            if feat!= 999:
                feat = feat / (self.eps + neighs_dist[0])
            feats.append(feat)
        
        assert len(feats) == self.n_classes
        return_list += [feats]
        
        '''
            5. 
               5.1 Distance to Kth neighbor
                   Think of this as of quantiles of a distribution
               5.2 Distance to Kth neighbor normalized by 
                   distance to the first neighbor
               
               feat_51, feat_52 are answers to 5.1. and 5.2.
               should be scalars
               
               Do not forget to add self.eps to denominator.
        '''
        for k in self.k_list:
            
            feat_51 = neighs_dist[k-1]
            feat_52 = neighs_dist[k-1] / (neighs_dist[0] + self.eps)
            
            return_list += [[feat_51, feat_52]]
        
        '''
            6. Mean distance to neighbors of each class for each K from `k_list` 
                   For each class select the neighbors of that class among K nearest neighbors 
                   and compute the average distance to those objects
                   
                   If there are no objects of a certain class among K neighbors, set mean distance to 999
                   
               You can use `np.bincount` with appropriate weights
               Don't forget, that if you divide by something, 
               You need to add `self.eps` to denominator.
        '''
        for k in self.k_list:
            numerator = np.zeros(self.n_classes)
            denominator = np.full(self.n_classes, self.eps)
            t = neighs_y[:k].max() + 1
            numerator[:t] = np.bincount(neighs_y[:k], weights=neighs_dist[:k])
            denominator[:t] = self.eps + np.bincount(neighs_y[:k])
            feats = np.where(numerator>0, numerator/denominator, 999)
            
            assert len(feats) == self.n_classes
            return_list += [feats]
        
        
        # merge
        knn_feats = np.hstack(return_list)
        
        return knn_feats

In [0]:
numeric_df = train_df.select_dtypes(exclude=['object'])
y_train = numeric_df.target
numeric_df = scale(numeric_df.drop('target', axis=1))

### Get feature for train

In [125]:
# We will use two metrics for KNN
for metric in ['minkowski', 'cosine']:
    print (metric)
    
    # Set up splitting scheme, use StratifiedKFold
    # use skf_seed and n_splits defined above with shuffle=True
    skf = StratifiedKFold(n_splits=n_splits, random_state=skf_seed, shuffle=True)
    
    # Create instance of our KNN feature extractor
    # n_jobs can be larger than the number of cores
    NNF = NearestNeighborsFeats(n_jobs=4, k_list=k_list, metric=metric)
    
    # Get KNN features using OOF use cross_val_predict with right parameters
    preds = cross_val_predict(NNF, numeric_df, y_train, cv=skf)
    
    # Save the features
    np.save('knn_feats_%s_train.npy' % metric, preds)

minkowski


/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


KeyError: ignored

### Get feature for test

In [126]:
for metric in ['minkowski', 'cosine']:
  print (metric)

  # Create instance of our KNN feature extractor
  NNF = NearestNeighborsFeats(n_jobs=1, k_list=k_list, metric='minkowski')

  # Fit on train set
  NNF.fit(numeric_df, y_train)

  # Get features for test
  test_knn_feats = NNF.predict(X_test)
  
  # Dump the features to disk
  np.save('knn_feats_%s_test.npy' % metric , test_knn_feats)

minkowski
cosine


In [93]:
numeric_df

array([[ 0.38612048, -0.10467775, -0.09576705, ...,  0.59945067,
         0.11041439,  0.19980637],
       [-2.58986523, -0.340922  , -0.09576705, ...,  1.61232234,
         0.11041439,  0.19980637],
       [ 0.38612048, -0.11412752, -0.09576705, ..., -0.41342101,
        -0.24610575, -0.25547279],
       ...,
       [ 0.38612048,  0.44340891, -0.09576705, ..., -0.41342101,
        -0.24610575, -0.25547279],
       [ 0.38612048, -0.55826671, -0.09576705, ..., -0.41342101,
        -0.24610575, -0.25547279],
       [ 0.38612048,  2.56015738, -0.09576705, ..., -0.41342101,
        -0.24610575, -0.25547279]])

In [89]:
numeric_df.head()

,target,html_page_cnt,text_rows_mean,is_mis_accepted_function,is_mis_rejected_function,is_mis_accepted_product,is_mis_rejected_product,is_mis_keyword,have_error,acc_f_rej_f_sim,acc_p_rej_p_sim,kwd_acc_p_sim,kwd_rej_p_sim,kwd_acc_f_sim,kwd_rej_f_sim,dif_kwd_acc_p_rej_p,dif_kwd_acc_j_rej_j,frc_acc_f,frc_rej_f,frc_acc_p,frc_rej_p,frc_acc_f_0,frc_rej_f_0,frc_acc_p_0,frc_rej_p_0,frc_acc_f_1,frc_rej_f_1,frc_acc_p_1,frc_rej_p_1,frc_acc_f_2,frc_rej_f_2,frc_acc_p_2,frc_rej_p_2
0,2,2,73,0,0,1,1,0,0,0.000000,1.0,0.000000,0.0,0.047619,0.044776,0.000000,0.002843,0.018265,0.011785,0.002597,0.004149,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.018349,0.011885,0.002597,0.004149
1,2,1,48,0,0,1,1,0,0,0.061224,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.019787,0.023569,0.002597,0.004149,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.019878,0.023769,0.002597,0.004149
2,1,2,72,0,0,1,1,1,0,0.117647,1.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.031963,0.045455,0.002597,0.004149,0.0,0.0,0.0,0.0,0.032061,0.045455,0.002646,0.004255,0.000000,0.000000,0.000000,0.000000
3,2,2,11,0,0,1,1,0,0,0.024390,1.0,0.000000,0.0,0.083333,0.000000,0.000000,0.083333,0.015221,0.023569,0.002597,0.004149,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.015291,0.023769,0.002597,0.004149
4,1,2,40,0,0,0,1,0,0,0.041667,0.0,0.064516,0.0,0.018519,0.018868,0.064516,-0.000349,0.010654,0.010101,0.036364,0.004149,0.0,0.0,0.0,0.0,0.010687,0.010101,0.037037,0.004255,0.000000,0.000000,0.000000,0.000000
